# MELODI Lite API Example Usage

In [2]:
import json
import operator
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from environs import Env

## Configure parameters

In [7]:
API_URL = "http://melodi-lite.mrcieu.ac.uk/api/"

requests.get(f"{API_URL}/status").json()

True

### Get enriched SemMedDB objects for a given search term

Starting from a PubMed search, return all SemMedDB subject-predicate-object triples, their enrcichment metrics and publication information 

In [20]:
query_term_list=['pcsk9']

endpoint = "/enrich/"
url = f"{API_URL}{endpoint}"
params = {
    "text": query_term_list,
}
response = requests.post(url, data=json.dumps(params))
res = response.json()
df = (
    pd.json_normalize(res[query_term[0]])
)
print(df)

                                                triple  \
0    PCSK9:INTERACTS_WITH:Low Density Lipoprotein R...   
1    Hydroxymethylglutaryl-CoA Reductase Inhibitors...   
2    PCSK9:STIMULATES:Low Density Lipoprotein Receptor   
3      PCSK9:INHIBITS:Low Density Lipoprotein Receptor   
4                             ezetimibe:INHIBITS:PCSK9   
..                                                 ...   
273  Low Density Lipoprotein Receptor:INHIBITS:Seri...   
274            PCSK9:PREDISPOSES:Myocardial Infarction   
275  Hydroxymethylglutaryl-CoA Reductase Inhibitors...   
276  Cardiovascular arteriosclerosis unspecified:CO...   
277                      COG2:NEG_INTERACTS_WITH:PCSK9   

                                       subject_name subject_type subject_id  \
0                                             PCSK9         gngm     255738   
1    Hydroxymethylglutaryl-CoA Reductase Inhibitors         orch   C0360714   
2                                             PCSK9         gngm  

### Overlapping terms

We can provide two lists of query terms and identify overlapping data

In [23]:
x=['MLH1','MSH2','MLH3','MSH6','PMS1','PMS2','APC']
y=['Hereditary non-polyposis colon cancer']

endpoint = "/overlap/"
url = f"{API_URL}{endpoint}"
params = {
    "x": x,
    "y": y,
}
response = requests.post(url, data=json.dumps(params))
res = response.json()
df = (
    pd.json_normalize(res['data'])
)
print(df)

                                         triple_x       subject_name_x  \
0     MLH1 gene|MLH1:COEXISTS_WITH:MSH2 gene|MSH2       MLH1 gene|MLH1   
1     MLH1 gene|MLH1:COEXISTS_WITH:MSH2 gene|MSH2       MLH1 gene|MLH1   
2     MLH1 gene|MLH1:COEXISTS_WITH:MSH2 gene|MSH2       MLH1 gene|MLH1   
3     MLH1 gene|MLH1:COEXISTS_WITH:MSH2 gene|MSH2       MLH1 gene|MLH1   
4     MLH1 gene|MLH1:COEXISTS_WITH:MSH2 gene|MSH2       MLH1 gene|MLH1   
...                                           ...                  ...   
1683               Anthocyanins:COEXISTS_WITH:APC         Anthocyanins   
1684                 Protein C:INTERACTS_WITH:APC            Protein C   
1685        Phosphotransferases:COEXISTS_WITH:APC  Phosphotransferases   
1686                       Factor Va:INHIBITS:APC            Factor Va   
1687            beta catenin:STIMULATES:Oncogenes         beta catenin   

     subject_type_x   subject_id_x     predicate_x   object_name_x  \
0              gngm  C0879389|4292   COEX